2. Pick a dictionary (or dictionaries) of your choice from the Harvard IV set, 
the Loughran-McDonald set, or some other of your choosing that you think may be relevant for the data you collected for Q4 of the previous problem set. Then conduct the following exercise:
(a) Use the two methods above to score each document in your data.
(b) Explore whether the scores diﬀer according to the meta data ﬁelds you gathered: for example, do diﬀerent speakers/sources/etc tend to receive a higher score than
others?
(c) Do the answers to the previous question depend on whether tf-idf weighting is applied or not? Why do you think there is (or is not) a diﬀerence in your answers?

In [1]:
import kickass as ks
import xlrd
import re
%matplotlib inline

ImportError: No module named 'xlrd'

In [ ]:
import json
fd = open("../data/project_successful_final_tech_sample.json", "r")

project_json = json.load(fd)


In [ ]:
ks_projects = ks.ProjectCollection(project_json, "../data/stopwords.txt")

In [ ]:
#Cleaning documents---tokenizing words, removing stopwords, and stemming
ks_projects.clean_docs(2)


In [ ]:
#import word list

wordlist_excel = xlrd.open_workbook("../data/H4Wordlist.xls")

In [ ]:
wl_sheet = wordlist_excel.sheet_by_index(0)
word_dict = dict()

def pre_process(word):
    word = re.sub("#\d+", "", word.lower())
    word = ks.nltk.PorterStemmer().stem(word)
    return word

for i in range(wl_sheet.ncols):
    word_dict[wl_sheet.col(i)[0].value] = [pre_process(cell.value.lower()) \
                                     for cell in wl_sheet.col(i)[1:] if cell.value != ""]

    

In [ ]:
#Evaluating the top 20 ranks based on word counts in different dictionaries
ks_sentiment_count = {key: ks_projects.rank_count(word_dict[key], 20) \
                      for key in word_dict.keys()}

In [ ]:
for key in ks_sentiment_count.keys():
    print key, ks_sentiment_count[key][0:5]


In [ ]:
#Evaluating the top 20 ranks based on tfidf in different dictionaries
ks_sentiment_tfifd = {key: ks_projects.rank_tfidf(word_dict[key], 20) \
                      for key in word_dict.keys()}

In [ ]:
for key in ks_sentiment_tfifd.keys():
    print key, ks_sentiment_tfifd[key][0:5]

Observation:
   Looking at the data closely we realise that there is a difference in rankings of documents between the count and the tfidf methods. This is to be expected because tfidf reweights the count frequencies by how frequently they appear in all documents. 
   
   Below we have highlighted the differences we came accross
   
   It appears that for our sample size of 100 the dictionaries of "Active" and "Quality"

In [ ]:
for key in ks_sentiment_count.keys():
    
    print "Dictionary " + key.upper()
    print "Ranking" + "\t \t" + "Count" + "\t \t" + "Tfidf"
    for i in range(len(ks_sentiment_count[key])):
        if ks_sentiment_count[key][i][0] != ks_sentiment_tfifd[key][i][0]:
            print(str(i+1)+ "\t \t" + str(ks_sentiment_count[key][i][0]) + "\t \t" + \
            str(ks_sentiment_tfifd[key][i][0]) )
    print "\n"

We try to break down the rankings based on if the project has higher than or lower than the median number of financial backers

In [ ]:
regex = re.compile("\d+")

med_backers_list = []
backers = dict()

for project in project_json:
    backer = int(regex.findall(project["no_backers"])[0])
    med_backers_list.append(backer)
    backers[project["id"]]= backer

med_backers = ks.np.median(med_backers_list)


#add indicator variable if num backers if greater than median
def med_ind(ranking):

    for key in ranking.keys():
        for project_doc in ranking[key]:
            if backers[project_doc[0]] > med_backers:
                project_doc.append("GreaterMedian")
            else:
                project_doc.append("LessMedian")
    return ranking

ks_sentiment_tfifd = med_ind(ks_sentiment_tfifd)
ks_sentiment_count = med_ind(ks_sentiment_count)

In [ ]:
import matplotlib.pyplot as plt

def compare_top_n(top, ranking):
    
    greater_than_list = []
    less_than_list = []

    for key in ranking.keys():
        greater_than = 0
        less_than = 0
        for project_doc in ranking[key][0:top]:
            if project_doc[2] == "GreaterMedian":
                greater_than+= 1
            else:
                less_than += 1
        greater_than_list.append(greater_than)
        less_than_list.append(less_than)
    return greater_than_list, less_than_list


#Plot to compare TOP 10 ranked results by median backers indicator
def plot_results(top, ranking, typer):
    
    fig, ax = plt.subplots()
    n_groups = len(ks_sentiment_count.keys())
    index = ks.np.arange(n_groups)
    greater_than_list, less_than_list = compare_top_n(top,ranking)
    bar_width = 0.35

    opacity = 0.4

    rects1 = plt.bar(index, greater_than_list, bar_width,
                     alpha=opacity,
                     color='b',
                     label='GreaterThan')

    rects2 = plt.bar(index + bar_width, less_than_list, bar_width,
                     alpha=opacity,
                     color='r',
                     label='LessThan')

    plt.xlabel('Dictionary')
    plt.ylabel('')
    plt.title('Freq by Median # Backers Indicator by Count for '+ typer)
    plt.xticks(index + bar_width, ranking.keys() )
    plt.legend()
    plt.tight_layout()

In [ ]:
plot_results(10, ks_sentiment_count, "Count Based Ranking")

We notice that when we compare the top 10 results breakdown of rankings based on if a project has more backers than the median is more or less even accross dictionaries except for the ACTIVE dictionary.

However, if we consider the top 20 results, the differences become more stark

In [ ]:
plot_results(20, ks_sentiment_count, "Count Based Ranking")

Here we can clearly see that projects with more backers tend to have more occurrences across all dictionaries.

In [ ]:
plot_results(10, ks_sentiment_tfifd, "TFIDF Based Ranking")
plot_results(20, ks_sentiment_tfifd, "TFIDF Based Ranking")

As expected while the counts of rankings based on no of backers does not change across ranking types, what does change is the order of ranking of the project documents that we have demonstrated using a table in the previous section.